In [2]:
import sys
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install --upgrade tensorflow
!{sys.executable} -m pip install pydub

In [3]:
#!git clone https://github.com/SurachanLiao/music_generative.git

In [4]:
import argparse, os
import librosa
import numpy as np
from sklearn.decomposition import NMF
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D
from keras.initializers import HeNormal
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_files(directory):
    # Stores list of file names
    file_names = []
    # traverse whole directory
    for root, dirs, files in os.walk(directory):
        # select file name
        for file in files:
            # check if file is wav file
            if file.endswith('.wav'):
                file_names.append(file)

    return file_names

def get_data(directory):
    file_names = get_files(directory)

    W_flat_list = []
    H_flat_list = []
    X_phase_list = []
    # Decompose music pieces
    for file in file_names:
        file_path = f"{directory}/{file}"
        # Get data from file
        x, sr = librosa.load(file_path, duration=5)
        S = librosa.stft(x)
        # Get magnitude and phase components of data
        X, X_phase = librosa.magphase(S)
        X_phase_list.append(X_phase)
        # Decompose X into W and H matrices using NMF
        num_comp = 20
        model = NMF(n_components = num_comp, init='random', random_state=0, max_iter=2000)
        W = model.fit_transform(X)
        H = model.components_
        # Flattens the matrices into vectors
        W_flat_list.append(W.flatten())
        H_flat_list.append(H.flatten())

    X_train = np.stack(W_flat_list)
    y_train = np.stack(H_flat_list)

    return X_train, y_train, X_phase_list

def save_NMF_data(X_train, y_train, X_phase_list, fp):
    np.savez(fp, X_train = X_train, y_train = y_train, 
            X_phase_list = X_phase_list)


# model = Sequential()
# model.add(Dense(units=128, input_dim=training_set.shape[1], activation="relu", kernel_initializer=HeNormal())) # 1st Hidden layer
# model.add(Dense(units=128, activation="relu", kernel_initializer=HeNormal())) # 2nd Hidden layer
# model.add(Dense(units=128, activation="relu", kernel_initializer=HeNormal())) # 3nd Hidden layer
# model.add(Dense(units=labels.shape[1], activation="relu")) # Output layer

# def train_model(X_train, y_train):
#   model = None  # TODO figure out model initialization code for ANN
#   model.fit(X_train, y_train)
#   return model

In [5]:
X_train, y_train, X_phase_list = get_data("music_generative")
#print(X_train.shape) #save_NMF_data(X_train, y_train, args.npyFILE)

In [6]:
y_train.shape

(91, 4320)

In [7]:
save_NMF_data(X_train, y_train, X_phase_list, "data")

In [8]:
data = np.load("data.npz")
X_train = data['X_train']
y_train = data['y_train']
X_phase_list = data['X_phase_list']

In [9]:
training_set = X_train
labels = y_train

In [10]:
labels.shape

(91, 4320)

In [11]:
# Create Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D
from keras.initializers import HeNormal
model = Sequential()
model.add(Dense(units=512, input_dim=training_set.shape[1], activation="relu", kernel_initializer=HeNormal())) # 1st Hidden layer
model.add(Dense(units=512, activation="relu", kernel_initializer=HeNormal())) # 2nd Hidden layer
model.add(Dense(units=512, activation="relu", kernel_initializer=HeNormal())) # 3nd Hidden layer
model.add(Dense(units=labels.shape[1], activation="relu")) # Output layer

In [12]:
# Compile Model
model.compile(optimizer='adam', loss="mean_squared_error", metrics=["accuracy"])

In [13]:
# Run Model
history = model.fit(training_set, labels, epochs=100, verbose=1, batch_size = 1)

Epoch 1/100
91/91 [==============================] - 5s 53ms/step - loss: 3.0351 - accuracy: 0.0000e+00
Epoch 2/100
91/91 [==============================] - 5s 55ms/step - loss: 3.0242 - accuracy: 0.0000e+00
Epoch 3/100
91/91 [==============================] - 6s 65ms/step - loss: 2.9970 - accuracy: 0.0220
Epoch 4/100
91/91 [==============================] - 5s 54ms/step - loss: 2.9449 - accuracy: 0.0220
Epoch 5/100
91/91 [==============================] - 4s 49ms/step - loss: 2.8885 - accuracy: 0.0440
Epoch 6/100
91/91 [==============================] - 5s 58ms/step - loss: 2.8836 - accuracy: 0.0440
Epoch 7/100
91/91 [==============================] - 5s 52ms/step - loss: 2.9728 - accuracy: 0.0879
Epoch 8/100
91/91 [==============================] - 5s 55ms/step - loss: 3.0118 - accuracy: 0.0769
Epoch 9/100
91/91 [==============================] - 6s 63ms/step - loss: 2.9937 - accuracy: 0.0659
Epoch 10/100
91/91 [==============================] - 5s 55ms/step - loss: 2.9073 - accuracy

91/91 [==============================] - 5s 57ms/step - loss: 2.2734 - accuracy: 0.3956
Epoch 83/100
91/91 [==============================] - 6s 63ms/step - loss: 2.2766 - accuracy: 0.4066
Epoch 84/100
91/91 [==============================] - 5s 59ms/step - loss: 2.2385 - accuracy: 0.3956
Epoch 85/100
91/91 [==============================] - 5s 56ms/step - loss: 2.2327 - accuracy: 0.3846
Epoch 86/100
91/91 [==============================] - 5s 56ms/step - loss: 2.2362 - accuracy: 0.3956
Epoch 87/100
91/91 [==============================] - 6s 61ms/step - loss: 2.2357 - accuracy: 0.4066
Epoch 88/100
91/91 [==============================] - 5s 59ms/step - loss: 2.2185 - accuracy: 0.4066
Epoch 89/100
91/91 [==============================] - 5s 58ms/step - loss: 2.2272 - accuracy: 0.3846
Epoch 90/100
91/91 [==============================] - 5s 57ms/step - loss: 2.2193 - accuracy: 0.3956
Epoch 91/100
91/91 [==============================] - 5s 57ms/step - loss: 2.2214 - accuracy: 0.3736
Epo

In [14]:
# Test model
# W_formatted = np.array([W_flattened])
#W_formatted = np.array([X_train])
H_predicted_flat = model.predict(X_train)

In [15]:
H_predicted_flat.shape

(91, 4320)

In [16]:
# Reformat H
H_predicted = H_predicted_flat[0].reshape(20, -1)

In [17]:
H_predicted.shape

(20, 216)

In [18]:
X = X_train[0]
X.reshape(-1, 20)

array([[2.88332701e-02, 1.27593115e-01, 2.38136551e-03, ...,
        1.44590260e-02, 1.83248834e-03, 0.00000000e+00],
       [0.00000000e+00, 1.30610541e-01, 3.20345769e-03, ...,
        5.02347481e-03, 0.00000000e+00, 8.09298828e-02],
       [1.14518702e-01, 2.82753080e-01, 4.90938760e-02, ...,
        0.00000000e+00, 0.00000000e+00, 4.64812905e-01],
       ...,
       [0.00000000e+00, 1.14350820e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.77858703e-04],
       [0.00000000e+00, 1.11855756e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.77725262e-04],
       [0.00000000e+00, 1.11447545e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.77909300e-04]], dtype=float32)

In [19]:
H = y_train[0].reshape(20, -1)
H.shape

(20, 216)

In [27]:
import soundfile as sf
S_reconstructed = X.reshape(-1, 20) @ H_predicted * X_phase_list[0]
x_reconstructed = librosa.istft(S_reconstructed)

S = X.reshape(-1, 20) @ H * X_phase_list[0]
x = librosa.istft(S)


sf.write("test0n-reconstructed.wav", x_reconstructed, 22050)
sf.write("test0n.wav", x, 22050)

In [28]:
from pydub import AudioSegment

song = AudioSegment.from_wav("test0n-reconstructed.wav")

# reduce volume by 10 dB
song_10_db_louder = song + 20

# but let's make him *very* quiet
#song = song - 36

# save the output
song_10_db_louder.export("louder.wav", "wav")

<_io.BufferedRandom name='louder.wav'>

In [22]:
save_NMF_data(X_train, y_train, args.npyFILE)

NameError: name 'args' is not defined